This notebook was used to compile all of the available data from the Utah Flux Network stations.  It should only need to be used once, as other notebooks are used to comile the newer data.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import geopandas as gpd
import urllib
import sys
import pathlib
import glob
import matplotlib.pyplot as plt
from urllib.parse import quote
from sqlalchemy import create_engine
import configparser


import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px





In [2]:
sys.path.append("//")
#sys.path.append("../../Micromet")
import micromet
from micromet import AmerifluxDataProcessor
%matplotlib inline

In [3]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

In [4]:
site_folders = {'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTN':'Juab'
                }


compdf = {}

am = micromet.AmerifluxDataProcessor()

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/')
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(raw_data, data_path="C:/Users/paulinkenbrandt/Documents/GitHub/MicroMet/data/extreme_values.csv")
        am_df = am_data.et_data
        compdf[key] = am_df

        am_df.to_csv(f"../../station_data/{key}_HH_{am_df['TIMESTAMP_START'].values[0]:}_{am_df['TIMESTAMP_END'].values[-1]:}.csv")
        #am_df['stationid'] = key
        
        #am_df.to_sql('amfluxeddy', key, engine, if_exists='append',schema='groundwater')
        
cdf = pd.concat(compdf,axis=0)
cdf.index.set_names(['stationid','datetime_start'],inplace=True)
#cdf.rename(columns={'level_0':'stationid'},inplace=True)
#cdf.to_parquet('../station_data/all_data.parquet')
for col in cdf.columns:
    cdf.rename(columns={col:col.lower()},inplace=True)


US-UTD
US-UTB
US-UTJ
US-UTW
US-UTE
US-UTM
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20211228\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20220215\8442_Flux_AmeriFluxFormat_0.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Matheson\d20220323\8442_Flux_AmeriFluxFormat_0.dat
US-UTP


C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:

US-CdM
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Cedar_mesa\e20241008\45027_Flux_AmeriFluxFormat_6.dat
US-UTV
US-UTN
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\20210622_farm\8441_Flux_AmeriFluxFormat_4.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\combined_farm\8441_Flux_AmeriFluxFormat_4.dat
Unknown header format for file: H:\UGS_Flux\Data_Downloads\Juab\20210826_farmNephi\20210826\8441_Flux_AmeriFluxFormat_11.dat


C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:945: RuntimeWarning: Mean of empty slice
  avg = np.nanmean(arr)
C:\Users\paulinkenbrandt\Anaconda3\envs\pygis12\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\paulinkenbrandt\Documents\GitHub\MicroMet\micromet\converter.py:

In [5]:
cdf.to_sql(name = 'amfluxeddy',
           schema='groundwater',
           con=engine,
           if_exists='replace',
           chunksize=2000)

15788

In [10]:
am_df[['file_no','TIMESTAMP_START','TIMESTAMP_END']]

,file_no,TIMESTAMP_START,TIMESTAMP_END
datetime_start,,,
2018-08-22 10:00:00,-9999.0,201808221000,201808221030
2018-08-22 10:30:00,8.0,201808221030,201808221100
2018-08-22 11:00:00,8.0,201808221100,201808221130
2018-08-22 11:30:00,8.0,201808221130,201808221200
2018-08-22 12:00:00,8.0,201808221200,201808221230
...,...,...,...
2024-05-09 08:30:00,8.0,202405090830,202405090900
2024-05-09 09:00:00,8.0,202405090900,202405090930
2024-05-09 09:30:00,8.0,202405090930,202405091000


In [48]:
from pathlib import Path

def file_compile(
    raw_fold: Path,
    station_folder_name: Path,
    search_str="*Flux_AmeriFluxFormat*.dat",
):
    """
    Compiles raw AmeriFlux datalogger files into a single dataframe.

    :param raw_fold: Path to the root folder of raw datalogger files
    :type raw_fold: pathlib.Path
    :param station_folder_name: Name of the station folder containing the raw datalogger files
    :type station_folder_name: str
    :return: Dataframe containing compiled AmeriFlux data, or None if no valid files found
    :rtype: pandas.DataFrame or None
    """
    amflux = {}
    station_folder = raw_fold / station_folder_name


    # iterate through specified folder of raw datalogger files (.dat); Match to AmeriFlux datalogger files
    for file in station_folder.rglob(search_str):
        # get the base number of the raw ameriflux file
        baseno = file.name.split(".")[0]

        try:
            file_number = int(baseno.split("_")[-1])
        except:
            file_number = 9999

        if file_number >= 0:
            try:
                df = pd.read_csv(file,
                                 skiprows=[1, 2],
                                 usecols=[0,1],
                                 na_values=[-9999,"NAN","NaN"])
            except:
                df = None
            if df is not None:
                #print(df.columns[0])
                # print(file)
                if df.columns[0] ==  "TIMESTAMP_START":
                    df["file_no"] = file_number
                    df["baseno"] = baseno
                    amflux[baseno] = df

    if amflux:
        # concat dataframes that were successfully read in
        et_data = pd.concat(amflux, axis=0).reset_index()
        #et_data = et_data.drop(columns=["level_0", "level_1"])
    else:
        et_data = None
    return et_data

date_data = {}
for key, value in site_folders.items():

    #print(key)
    raw_fold = pathlib.Path('H:/UGS_Flux/Data_Downloads/')
    date_data[key] = file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")

df = pd.concat(date_data,axis=0)
dfmod = df.drop(['level_0','level_1'],axis=1).reset_index().drop(['level_1'],axis=1).rename(columns={'level_0':'stationid'})#.set_index(['stationid','file_no'])
dfmod['TIMESTAMP_START'] = dfmod['TIMESTAMP_START'].astype('int')
dfmod['TIMESTAMP_END'] = dfmod['TIMESTAMP_END'].astype('int')
dfmod.sort_values(['stationid','file_no'],inplace=True)

def f(x):
    d = {}
    d['start_min'] = x['TIMESTAMP_START'].min()
    d['start_max'] = x['TIMESTAMP_START'].max()
    d['end_min'] = x['TIMESTAMP_END'].min()
    d['end_max'] = x['TIMESTAMP_END'].max()

    d['file'] = x['baseno'].values[0]
    return pd.Series(d, index=d.keys())

dstats = dfmod.groupby(['stationid','file_no']).apply(f)


C:\Users\paulinkenbrandt\AppData\Local\Temp\1\ipykernel_11876\1203840080.py:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dstats = dfmod.groupby(['stationid','file_no']).apply(f)


180